In [80]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Activation, Dense
from keras import optimizers
from keras.layers import Dropout

In [81]:
iris = load_iris()

In [82]:
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [83]:
data = pd.DataFrame(iris['data'], columns = iris['feature_names'] )
target = pd.DataFrame(iris['target'],columns = ['target'])

In [84]:
#combine the input predictors and target so that it can be split into training and testing
data_target = data.join(target)

In [85]:
data_target.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [86]:
data_target['target'].value_counts()

2    50
1    50
0    50
Name: target, dtype: int64

In [87]:
#for this implementaion, let's make the problem a binary. So considering only '0' and '1' as the target
X = np.array(data_target[(data_target['target'] == 0) | (data_target['target'] == 1)].drop('target', axis=1))
y = np.array(data_target[(data_target['target'] == 0) | (data_target['target'] == 1)]['target']).reshape(100,1)

In [88]:
train_x,test_x,train_y,test_y = train_test_split(X,y,test_size = 0.2)

print('Shape of train_x:', train_x.shape)
print('Shape of train_y:', train_y.shape)
print('Shape of test_x:', test_x.shape)
print('Shape of test_y:', test_y.shape)

Shape of train_x: (80, 4)
Shape of train_y: (80, 1)
Shape of test_x: (20, 4)
Shape of test_y: (20, 1)


Splitting the data into 9 mini batches

In [89]:
batches = 9

#get the total number of batches
batch_count = train_x.shape[0] // batches

batch_trainx = []
batch_trainy = []

for i in range(0, batch_count):    
    begin = i * batches
    end = (i + 1) * batches

    batch_trainx.append(train_x[begin:end])
    batch_trainy.append(train_y[begin:end])

#when the total count is not exactly divisible by batches
left_out = train_x.shape[0] % batches

if left_out != 0:    
    batch_trainx.append(train_x[end: end + left_out])
    batch_trainy.append(train_y[end: end + left_out])

In [90]:
# intializing the weights for each layer
def intialize_weights(layer_dims):
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)
    
    #initialise the value of weights based on the number of layers
    for i in range(1,L-1):
        #for the hidden layers we will use He initialisation because of relu activation
        parameters['W'+str(i)] = np.random.randn(layer_dims[i-1],layer_dims[i]) * np.sqrt(2/layer_dims[i-1])
        parameters['v'+str(i)] = np.zeros([layer_dims[i-1],layer_dims[i]])
        parameters['b'+str(i)] = np.zeros([1, layer_dims[i]])   
        parameters['u'+str(i)] = np.zeros([1, layer_dims[i]])   
        
            
    #for the last layer we can use Xavier initialisation 
    parameters['W' + str(i+1)] = np.random.randn(layer_dims[i],layer_dims[i+1]) * np.sqrt(1/layer_dims[i])
    parameters['v' + str(i+1)] = np.zeros([layer_dims[i],layer_dims[i+1]])
    parameters['b'+str(i+1)] = np.zeros([1, layer_dims[i+1]])    
    parameters['u'+str(i+1)] = np.zeros([1, layer_dims[i+1]])    
    
    return parameters

In [91]:
#forward propagation
def forward_propagation(layer_dims,train_x,parameters):
    
    caches = []
    Aprev = train_x
    L = len(layer_dims)
    
    #forward propagation for all the layers except last layer
    for i in range(1,L-1): 
        W = parameters['W'+ str(i)]
        b = parameters['b' + str(i)] 
        Z = np.dot(Aprev, W) + b  
        Aprev = np.maximum(0,Z)       
        cache = Aprev, W, b
        caches.append(cache)     
    
    #forward propagation for the last layer
    W = parameters['W'+ str(L-1)]
    b = parameters['b' + str(L-1)]
    Zlast = np.dot(Aprev, W) + b    
    Alast = 1/(1 + np.exp(-Zlast))   
    cache = Alast, W, b

    caches.append(cache)
    return caches

In [92]:
#cost function calculation
def cost_calculate(predict_y,train_y):
    m = train_y.shape[0]
    cost = -(np.dot(train_y.T, np.log(predict_y)) + np.dot((1-train_y).T, np.log(1-predict_y)))/m
    return cost

In [104]:
def backward_propagation(layer_dims, caches, parameters, train_x, train_y, learning_rate, beta, iteration):
    #backward propagation for the last layer
    #Extract the last array from the caches, as this corresponds to the final output
    L = len(caches)    
    Acurr,Wcurr,bcurr = caches[L - 1]  
    Aprev,Wprev,bprev = caches[L - 2]
    v = parameters['v'+str(L)]
    u = parameters['u'+str(L)]

    m = train_y.shape[0]   
    
    dzprev = (Acurr - train_y)    
    dwlast = np.dot(Aprev.T, dzprev)/m    
    dblast = np.sum(dzprev, keepdims = True, axis = 0)/m   
    dvlast = (beta * v) + ((1-beta) * dwlast)/(1-np.power(beta, iteration))
    dulast = (beta * u) + ((1-beta) * dblast)/(1-np.power(beta, iteration))
    
    parameters['W' + str(L)]= parameters['W' + str(L)] - (learning_rate * dvlast)    
    parameters['b' + str(L)]= parameters['b' + str(L)] - (learning_rate * dulast)   
    parameters['v' + str(L)]= dvlast
    parameters['u' + str(L)]= dulast
            
    for i in reversed(range(L-1)):
        Anext,Wnext,bnext = caches[i+1]
        Acurr,Wcurr,bcurr = caches[i]  
        v = parameters['v'+str(i+1)]
        u = parameters['u'+str(i+1)]
        
        if i == 0:
            Aprev = train_x
        else:            
            Aprev,Wprev,bprev = caches[i-1]
                
        dzcurr = np.where(Acurr > 0,1,Acurr)                     
        dzprev = np.multiply(np.dot(dzprev,Wnext.T), dzcurr)
        
        dW = np.dot(Aprev.T,dzprev)/m
        db = np.sum(dzprev, keepdims = True, axis = 0)/m  
        dv = (beta * v) + ((1-beta) * dW)/(1-np.power(beta, iteration))
        du = (beta * u) + ((1-beta) * db)/(1-np.power(beta, iteration))
        
        parameters['W' + str(i+1)]= parameters['W' + str(i+1)] - (learning_rate * dv)
        parameters['b' + str(i+1)]= parameters['b' + str(i+1)] - (learning_rate * du)    
        parameters['v' + str(i+1)]= dv
        parameters['u' + str(i+1)]= du
        
    return parameters

In [105]:
def complete_model(layer_dims, batch_trainx, batch_trainy, learning_rate, iterations, beta):
        
    L = len(layer_dims)
    # Intialize the weights
    parameters = intialize_weights(layer_dims)
    act_iteration = 0
    
    for i in range(iterations):        
        for j in range(len(batch_trainx)):            
            #forward propagation
            caches = forward_propagation(layer_dims,batch_trainx[j],parameters)
        
            #calculate the cost 
            A,W,b = caches[-1]
            cost = cost_calculate(A,batch_trainy[j])
            act_iteration += 1
            parameters = backward_propagation(layer_dims, caches, parameters,batch_trainx[j], batch_trainy[j], learning_rate, beta, act_iteration)
            
        if i%1000 == 0:
            print('The cost after iteration {}: {}'.format(i, np.squeeze(cost)))
        #backward propagation            

    return parameters

In [106]:
layer_dims = [4,5,3,1]
learning_rate = 0.15
iterations = 5000
beta = 0.9
parameters = complete_model(layer_dims, batch_trainx, batch_trainy, learning_rate, iterations, beta)

The cost after iteration 0: 0.37198811438904533
The cost after iteration 1000: 0.000733552926585688
The cost after iteration 2000: 0.00036638842016319107
The cost after iteration 3000: 0.000244082220490538
The cost after iteration 4000: 0.00018296735587746365


In [96]:
caches = forward_propagation(layer_dims,test_x,parameters)
A,W,b = caches[-1]
cost = cost_calculate(A, test_y)
cost


array([[0.00013839]])

#### SGD using keras implementation

In [99]:
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras import optimizers
from keras.layers import BatchNormalization, Dropout

In [101]:
model = Sequential()
model.add(Dense(50, input_shape = (4, ),kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Dense(50,kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Dense(1,kernel_initializer='glorot_normal'))
model.add(Activation('sigmoid'))

In [102]:
sgd = optimizers.SGD(lr = 0.001)
model.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [103]:
model.fit(train_x, train_y, batch_size=9, validation_data=(test_x, test_y), epochs=50)

Train on 80 samples, validate on 20 samples
Epoch 1/50
80/80 [==============================] - 0s 2ms/step - loss: 1.3097 - accuracy: 0.5125 - val_loss: 0.9812 - val_accuracy: 0.4500
Epoch 2/50
80/80 [==============================] - 0s 112us/step - loss: 0.8018 - accuracy: 0.5125 - val_loss: 0.7112 - val_accuracy: 0.4500
Epoch 3/50
80/80 [==============================] - 0s 138us/step - loss: 0.6019 - accuracy: 0.5500 - val_loss: 0.5308 - val_accuracy: 1.0000
Epoch 4/50
80/80 [==============================] - 0s 150us/step - loss: 0.4702 - accuracy: 0.9625 - val_loss: 0.4296 - val_accuracy: 1.0000
Epoch 5/50
80/80 [==============================] - 0s 163us/step - loss: 0.3823 - accuracy: 1.0000 - val_loss: 0.3613 - val_accuracy: 1.0000
Epoch 6/50
80/80 [==============================] - 0s 150us/step - loss: 0.3234 - accuracy: 1.0000 - val_loss: 0.3115 - val_accuracy: 1.0000
Epoch 7/50
80/80 [==============================] - 0s 138us/step - loss: 0.2802 - accuracy: 1.0000 - val_